In [ ]:
!pip install jiwer --quiet
import duckdb
import pandas as pd
import os

#os.chdir("../../../")
%load_ext sql
conn = duckdb.connect("duckdb/main.duckdb")
write_conn = duckdb.connect("duckdb/main.duckdb")  # For UPDATEs and COPY
%sql conn --alias duckdb


%config SqlMagic.displaylimit = 50

In [ ]:
%%sql
select  * FROm block_quality where correct_ind IN('Y','C') and len(newspaper) < 10 

In [ ]:
%%sql
/* Check for newspapers that do not have text from current model */
select count(*) from(
SELECT PV.Newspaper, PV.Block , PV.Text
FROM Full_Text PV
LEFT JOIN Full_Text NV
ON NV.Newspaper = PV.Newspaper
AND NV.Block = PV.Block
AND NV.Source = 'gpt-4o'
AND NV.Type = 'block_prompt'
LEFT JOIN Full_Text NVGem
ON NVGem.Newspaper = PV.Newspaper
AND NVGem.Block = PV.Block
AND NVGem.Source = 'gemini-2.5-pro'
AND NVGem.Type = 'block_prompt'
WHERE NV.Text IS NULL
AND NVGem IS NULL
AND PV.Source = 'split_blocks.py'
AND PV.Type = 'N/A'
AND len(PV.Text) > 100 
AND NOT EXISTS(
SELECT 1 FROM BLOCK_QUALITY BQ
WHERE BQ.NEwspaper = PV.newspaper
AND left(BQ.block,7) = PV.block
AND Bq.Correct_Ind  = 'Y'
 )) a

In [ ]:
conn.close()
write_conn.close()

In [ ]:
%%sql
--SELECT newspaper, idx, CER, WER, Per_word_error FROM val_text

select Newspaper, CAST(mean(cer) AS DECIMAL(18,2)) CER , CAST(mean(wer) AS DECIMAL(18,2)) WER, CAST(mean(per_word_error) AS DECIMAL(18,2))BoW from val_text 

--where newspaper <>  '3B_Model_Plus1000.json' 
group by 1

In [ ]:
%%sql
select * from full_text where
1=1
--AND block = 'TB00012'
--AND Newspaper = 'postimeesew19371212.1.2'
and  text like '%Nõnda et annetustena on tulnud hinge pealt%' 
--and block <> 'All'

In [ ]:
%%sql

select 'Newspapers' as Type, Count(*) as Amt FROM(select distinct newspaper FROM full_text where source = 'dea.digar')A 
union all
select 'Text_Length' as Type, sum(text) as Amt FROM(select len(text) as text FROM full_text where source = 'dea.digar' )A 
union all
select 'Final text' , sum(ln) FROM(select len(train_input) - 8 /* The coordinates in the end */ as ln  FROM OLMOCR_train ) a
union all
select 'Final Blocks' , count(*) FROM(select distinct newspaper,block FROM OLMOCR_train ) a
union all
 select 'Final Newspapers' , count(*) FROM(select distinct newspaper FROM OLMOCR_train ) a

In [ ]:
%%sql
SElect count(*)  FROM full_text ft
where source IN('split_blocks_no_coords')
and not exists(
    select 1 from OLMOCR_train OT
    where ot.newspaper = ft.newspaper
)
--and len(text) > 100
limit 2

In [ ]:
%%sql
select *From OLMOCR_train where newspaper ='sakalaew19361118.1.6' and block = 'TB00004'

In [ ]:
%%sql
--select count(*) FROM(select distinct newspaper, block FROM full_text where Source = 'gemini-2.5-pro' and Type = 'block_prompt')
select count(*) FROM(select distinct newspaper, block FROM full_text where Source = 'gemini-2.5-pro' and Type = 'block_prompt')

In [ ]:

%%sql
select count(*) FROM(
SELECT PV.Newspaper, PV.Block , PV.Text, nvgem.*
FROM Full_Text PV
LEFT JOIN Full_Text NV
ON NV.Newspaper = PV.Newspaper
AND NV.Block = PV.Block
AND NV.Source = 'gpt-4o'
AND NV.Type = 'block_prompt'
LEFT JOIN Full_Text NVGem
ON NVGem.Newspaper = PV.Newspaper
AND NVGem.Block = PV.Block
AND NVGem.Source = 'gemini-2.5-pro'
AND NVGem.Type = 'block_prompt'

WHERE 
1=1
AND NV.Text IS NULL
AND NVGem.Text IS NULL
AND PV.Source = 'split_blocks.py'
AND PV.Type = 'N/A'
AND len(PV.Text) > 100 
AND  NOT EXISTS(
SELECT 1 FROM BLOCK_QUALITY BQ
WHERE BQ.NEwspaper = PV.newspaper
AND left(BQ.block,7) = PV.block
AND Bq.Correct_Ind = 'Y' 
) )a
--and pv.block = 'TB00001' and pv.newspaper = 'eestikirikeelk19260527.1.2'

In [ ]:
%%sql
select count(*) FROM(

SELECT 
Newspaper,
Block,
MAX(CASE WHEN Source = 'split_blocks_pypdf' then trim(text) else null END)  AS Train_Input,
MAX(CASE WHEN Source IN( 'split_blocks_no_coords') then trim(text) else null END)  AS Split_blocks_No_Coords
FROM Full_Text
where source  IN( 'split_blocks_pypdf','split_blocks_no_coords')
AND Block <> 'All'
AND EXISTS(
SELECT 1 FROM BLOCK_QUALITY BQ
WHERE BQ.NEwspaper = Full_Text.newspaper
AND left(BQ.block,7) = full_text.block
AND Bq.Correct_Ind = 'Y' 
)
GROUP BY Newspaper,Block
having Train_input is not null and Split_blocks_No_Coords is not null
 ) a

In [ ]:
%%sql
SELECT distinct newspaper  FROM val_text-- where newspaper = '3B_Model_Plus1000.json'  and cer >= 0.9 order by cer desc limit 11

In [ ]:
%%sql

select 
Newspaper,
sum(case when cer <= 0.01 then 1 else 0 end ) GOOD,
max(case when cer <= 0.01 then len(reference) - len(response) else null end ) GOOD_LEN_DIFF,
avg(case when cer <= 0.01 then len(reference) END ) AS good_len_avg,
sum(case when cer <= 0.05 and cer > 0.01  then 1 else 0 end ) OK,
max(case when cer <= 0.05 and cer > 0.01  then len(reference) - len(response) else null end ) OK_LEN_DIFF,
avg(case when cer <= 0.05 and cer > 0.01 then len(reference) END ) AS ok_len_avg,
sum(case when cer > 0.05 then 1 else 0 end ) NOK,
max(case when cer > 0.05 then len(reference) - len(response) else null end ) NOK_LEN_DIFF,
mean(case when cer > 0.05 then len(reference) END ) AS nok_len_avg,
sum(case when cer >= 0.5 then 1 else 0 end ) BAD,
max(case when cer >= 0.5  then len(reference) - len(response) else null end ) BAD_LEN_DIFF,
mean(case when cer >= 0.5 then len(reference) END ) AS bad_len_avg,
sum(case when cer >= 0.9 then 1 else 0 end ) Horrible,
max(case when cer >= 0.9 then len(reference) - len(response) else null end ) HORRIBLE_LEN_DIFF,
mean(case when cer >= 0.9 then len(reference) END ) AS horrible_len_avg


 from val_Text 
 where newspaper = '3B_Model_Plus1000.json' 
 group by 1
  limit 5

In [ ]:
%%sql

select *FROM(
select 
CASE WHEN Newspaper = 'QWEN_3B_Base.json' THEN 'Base ' || Category ELSE 'Tuned ' ||  Category end AS Category,
Count as Count,
CAST(abs(Len_Diff) AS  INTEGER) Len_Diff ,
CAST(Len_Avg AS DECIMAL(18,1)) Len_Avg
FROM(

SELECT 
       Newspaper,
       '<1%' AS category, 
        1 rnk,
       SUM(CASE WHEN cer <= 0.01 THEN 1 ELSE 0 END) AS count,
       MAX(CASE WHEN cer <= 0.01 THEN len(reference) - len(response) ELSE NULL END) AS len_diff,
       AVG(CASE WHEN cer <= 0.01 THEN len(reference) END) AS len_avg
FROM val_Text
WHERE Newspaper IN('QWEN_3B_Base.json','3B_Model.json') 
GROUP BY Newspaper

UNION ALL

SELECT
       Newspaper,
 '<5%' AS category, 
        2,
       SUM(CASE WHEN cer <= 0.05 AND cer > 0.01 THEN 1 ELSE 0 END) AS count,
       MAX(CASE WHEN cer <= 0.05 AND cer > 0.01 THEN len(reference) - len(response) ELSE NULL END) AS len_diff,
       AVG(CASE WHEN cer <= 0.05 AND cer > 0.01 THEN len(reference) END) AS len_avg
FROM val_Text
WHERE Newspaper IN('QWEN_3B_Base.json','3B_Model.json') 
GROUP BY Newspaper

UNION ALL

SELECT 
       Newspaper,
'>5%' AS category, 
3   ,
       SUM(CASE WHEN cer > 0.05 AND cer < 0.5  THEN 1 ELSE 0 END) AS count,
       MAX(CASE WHEN cer > 0.05 AND cer < 0.5 THEN len(reference) - len(response) ELSE NULL END) AS len_diff,
       AVG(CASE WHEN cer > 0.05 AND cer < 0.5 THEN len(reference) END) AS len_avg
FROM val_Text
WHERE Newspaper IN('QWEN_3B_Base.json','3B_Model.json') 
GROUP BY Newspaper

UNION ALL

SELECT 
       Newspaper,
'>50%' AS category, 
   4,
       SUM(CASE WHEN cer >= 0.5 AND cer < 0.9 THEN 1 ELSE 0 END) AS count,
       MAX(CASE WHEN cer >= 0.5 AND cer < 0.9 THEN len(reference) - len(response) ELSE NULL END) AS len_diff,
       AVG(CASE WHEN cer >= 0.5 AND cer < 0.9 THEN len(reference) END) AS len_avg
FROM val_Text
WHERE Newspaper IN('QWEN_3B_Base.json','3B_Model.json') 
GROUP BY Newspaper

UNION ALL

SELECT
       Newspaper,
 '>90%' AS category, 
5,
       SUM(CASE WHEN cer >= 0.9 THEN 1 ELSE 0 END) AS count,
       MAX(CASE WHEN cer >= 0.9 THEN len(reference) - len(response) ELSE NULL END) AS len_diff,
       AVG(CASE WHEN cer >= 0.9 THEN len(reference) END) AS len_avg
FROM val_Text
WHERE Newspaper IN('QWEN_3B_Base.json','3B_Model.json') 
GROUP BY Newspaper) a  order by rnk asc) a order by category asc

In [ ]:
conn.close()
write_conn.close()

In [ ]:
silver_root = "src/datasets/silver/scanned"

for paper_name in os.listdir(silver_root):
    paper_path = os.path.join(silver_root, paper_name)
    textfile_path = os.path.join(paper_path, "text", "continuous_text.txt")
    if not os.path.isfile(textfile_path):
        continue

    with open(textfile_path, "r", encoding="utf-8") as f:
        textfile_content = "".join(f.readlines())

    # Insert only if this newspaper doesn't already exist with the same source/type
    write_conn.execute("""
        INSERT INTO Full_Text (Newspaper, Source, Type, Text, Timestamp)
        SELECT ?, 'dea.digar', 'corrected', ?, CURRENT_TIMESTAMP
        WHERE NOT EXISTS (
            SELECT 1 FROM Full_Text
            WHERE Newspaper = ?
              AND Source = 'dea.digar'
              AND Type = 'corrected'
        )
        """, (paper_name, textfile_content, paper_name))


In [ ]:
%%sql
SELECT *   FROM Full_Text
where
1=1
AND newspaper = 'sakalaew19361118.1.6'
and Source = 'dea.digar'

In [ ]:
%%sql
CREATE TABLE IF NOT EXISTS val_text (
    Newspaper TEXT,
    Source TEXT,
    Type TEXT,
    CER DOUBLE,
    WER DOUBLE,
    Per_Word_Error INTEGER,  
    Timestamp TIMESTAMP
);

In [ ]:
write_conn.execute("""
        COPY Validation TO 'duckdb/validation.parquet' (FORMAT PARQUET);
        """)

# LLM diff

In [ ]:
from jiwer import wer
from jiwer import cer
import re
from collections import Counter
def normalize(text):
    # Fix hyphenation at end of line (e.g., "soo-\nmaline" → "soomaline")
    text = re.sub(r'-\s*\n', '', text)

    # Replace " - " or other mid-sentence hyphens with space (e.g., "miski - muu" → "miski muu")
    text = re.sub(r'\s*[-–—]\s*', ' ', text)

    # Normalize w → v
    text = text.replace('w', 'v').replace('W', 'V')

    # Remove multiple newlines and whitespace
    text = re.sub(r'\s+', ' ', text)

    return text.strip()

def count_words(text):
    text = re.sub(r'[^\w\s]', '', text)
    return Counter(text.split())

def dict_diff(dict1, dict2):
    diff = {}
    # Keys only in dict1
    for k in dict1:
        if k not in dict2:
            diff[k] = ('only_in_orig', dict1[k])
        elif dict1[k] != dict2[k]:
            diff[k] = ('different_values', dict1[k], dict2[k])
    # Keys only in dict2
    for k in dict2:
        if k not in dict1:
            diff[k] = ('only_in_new', dict2[k])
    return diff

In [ ]:
from jiwer import wer
from jiwer import cer
import re
from collections import Counter

def normalize_ocr_text(text):
    # Step 1: Remove hyphenation across lines (e.g. wa-\nrastati -> warastati)
    text = re.sub(r'(\w+)-\n(\w+)', r'\1\2', text)

    # Step 2: Replace newlines with spaces
    text = re.sub(r'\s*\n\s*', ' ', text)

    # Step 3: Normalize spacing
    text = re.sub(r'\s+', ' ', text).strip()

    return text

cursor = conn.execute(
    """
SELECT 
Newspaper,
Block,
MAX(CASE WHEN Source IN( 'split_blocks_no_coords' )then trim(text) else null END)  AS Manual,
MAX(CASE WHEN Source IN( 'gemini-2.5-pro') then trim(text) else null END)  AS Pro,
MAX(CASE WHEN Source IN( 'gpt-4o') then trim(text) else null END)  AS GPT,
MAX(CASE WHEN Source IN( 'split_blocks_pypdf_no_coord' )then trim(text) else null END) AS Pypdf
FROM Full_Text
where source  IN( 'gemini-2.5-pro','gpt-4o','split_blocks_no_coords','split_blocks_pypdf_no_coord')
AND Block <> 'All'
AND  EXISTS(
SELECT 1 FROM BLOCK_QUALITY BQ
WHERE BQ.NEwspaper = Full_Text.newspaper
AND left(BQ.block,7) = full_text.block
AND Bq.Correct_Ind = 'Y' 
)
GROUP BY Newspaper,Block
having len(gpt) > 0  and  len(pro) > 0 is not null 
""")

row = cursor.fetchone()
results = []


while row:
    newspaper,block,orig, pro, gpt, pypdf = row
    
    # Remove some unimportant difference, might not be needed for untrained data
    #original_digar = normalize(orig) 
    #pro_norm = normalize(pro) 
    #gpt_norm = normalize(gpt)
    original_digar = orig
    pro_norm = pro
    gpt_norm = gpt
    pypdf_norm = pypdf

    if not pypdf_norm:
        errorc_pypdf = 9999
        errorc_pypdf_modern = 9999
    if pypdf_norm:
        modified_pypder = pypdf_norm.replace("w", "v").replace("W", "V")
        modified_pypdf = normalize_ocr_text(pypdf_norm)
        errorc_pypdf = cer(original_digar, pypdf_norm)
        errorc_pypdf_modern = cer(modified_orig, modified_pypdf)

    # Modernize w -> v
    modified_gpt= gpt_norm.replace("w", "v").replace("W", "V")
    modified_pro= pro_norm.replace("w", "v").replace("W", "V")
    modified_orig = original_digar.replace("w", "v").replace("W", "V")
    
    
    modified_gpt = normalize_ocr_text(modified_gpt)
    modified_pro = normalize_ocr_text(modified_pro)
    modified_orig = normalize_ocr_text(modified_orig)

    
    errorc_pro = cer(original_digar, pro_norm)
    errorc_gpt = cer(original_digar, gpt_norm)
    
    errorc_pro_modern = cer(modified_orig, modified_pro)
    errorc_gpt_modern = cer(modified_orig, modified_gpt)
    #print(errorc_pro)
    #print(errorc_gpt)
    


    df = pd.DataFrame ([{
            "newspaper": newspaper,
            "block" : block,
            "orig": original_digar,
            "Gemini": pro_norm,
            "GPT": gpt_norm,
            "cer_pro": errorc_pro,
            "ceer_gpt": errorc_gpt,
            "ceer_pydpf": errorc_pypdf,
            "ceer_mod_gpt": errorc_gpt_modern,
            "ceer_mod_pro": errorc_pro_modern,
            "ceer_mod_pypdf" : errorc_pypdf_modern,
        }])

    results.append({
        "newspaper": newspaper,
        "block": block,
        "cer_pro": errorc_pro,
        "cer_gpt": errorc_gpt,
        "ceer_pydpf": errorc_pypdf,
        "ceer_mod_gpt": errorc_gpt_modern,
        "ceer_mod_pro": errorc_pro_modern,
        "ceer_mod_pypdf" : errorc_pypdf_modern,
        "orig": original_digar,
        "Gemini": pro_norm,
        "GPT": gpt_norm,
        "pypdf": pypdf_norm
    })
    
    row = cursor.fetchone()

df = pd.DataFrame(results)



In [ ]:
write_conn.register("df", df)

In [ ]:
%%sql
SELECT 
sum(case when  ceer_mod_pro >= 0.1 then 1 else 0 end ) mt_01,
sum(case when  ceer_mod_pro > 0.05 and ceer_mod_pro < 0.1 then 1 else 0 end ) mt_005,
sum(case when  ceer_mod_pro <= 0.1 then 1 else 0 end ) lt_005,
sum(case when  ceer_mod_gpt >= 0.1 then 1 else 0 end ) mt_01,
sum(case when  ceer_mod_gpt > 0.05 and ceer_mod_gpt < 0.1 then 1 else 0 end ) mt_005,
sum(case when  ceer_mod_gpt <= 0.1 then 1 else 0 end ) lt_005,
avg(cer_pro),
avg(cer_gpt),
avg(ceer_mod_pro),
avg(ceer_mod_gpt)
    from df --order by cer_diff desc

In [ ]:
%%sql

select
--regexp_extract(newspaper, '^(.*?)(?=\d{8})') AS publication,
newspaper,
CAST(mean(ceer_pydpf) AS DECIMAL(18,2)) as unm_ceer
--,mean(ceer_mod_pypdf)   as m_ceer

FROM df where ceer_pydpf <> 9999 and ceer_mod_pypdf <> 9999
group by newspaper
order by unm_ceer desc

--where ceer_pydpf > 1.0 limit 5


In [ ]:
%%sql
select * From Full_Text where source ='split_blocks_pypdf_no_coord'

In [ ]:
%%sql
select count(*) FROM(
SELECT 
case when ceer_mod_pro <= 0.05 and ceer_mod_gpt >  0.05 then 1 else 0 end  as GPT_Negative,
case when ceer_mod_pro > 0.05 and ceer_mod_gpt <=  0.05 then 1 else 0 end  as Pro_Negative,
--ceer_mod_pro,
--ceer_mod_gpt,
--abs(ceer_mod_gpt - ceer_mod_pro) as diff,
--df.*
    from df
  where  
 -- ((ceer_mod_pro <= 0.05 and ceer_mod_gpt >  0.05 ) OR (ceer_mod_pro > 0.05 and ceer_mod_gpt <=  0.05))


   (ceer_mod_pro <= 0.05 OR ceer_mod_gpt <= 0.05)
  )a
  --   order by diff desc

In [ ]:
%%sql
select *  FROM full_text where source IN('gemini-2.5-pro','gpt-4o','split_blocks_no_coords','split_blocks.py','dea.digar') and newspaper ='kaja19350917-1.1.7' and (block = 'TB00046' )

## LLM diff no community indicator

In [ ]:
from jiwer import wer
from jiwer import cer
import re
from collections import Counter

def normalize_ocr_text(text):
    # Step 1: Remove hyphenation across lines (e.g. wa-\nrastati -> warastati)
    text = re.sub(r'(\w+)-\n(\w+)', r'\1\2', text)

    # Step 2: Replace newlines with spaces
    text = re.sub(r'\s*\n\s*', ' ', text)

    # Step 3: Normalize spacing
    text = re.sub(r'\s+', ' ', text).strip()

    return text

cursor = conn.execute(
    """
SELECT 
Newspaper,
Block,
MAX(CASE WHEN Source IN( 'split_blocks_no_coords' )then trim(text) else null END)  AS Manual,
MAX(CASE WHEN Source IN( 'gemini-2.5-pro') then trim(text) else null END)  AS Pro,
MAX(CASE WHEN Source IN( 'gpt-4o') then trim(text) else null END)  AS GPT
FROM Full_Text
where source  IN( 'gemini-2.5-pro','gpt-4o','split_blocks_no_coords')
AND Block <> 'All'
AND  NOT EXISTS(
SELECT 1 FROM BLOCK_QUALITY BQ
WHERE BQ.NEwspaper = Full_Text.newspaper
AND left(BQ.block,7) = full_text.block
AND Bq.Correct_Ind = 'Y' 
)
GROUP BY Newspaper,Block
HAVING (gpt IS NOT NULL AND length(gpt) > 0) OR (pro IS NOT NULL AND length(pro) > 0)
""")

row = cursor.fetchone()
results = []


while row:
    newspaper,block,orig, pro, gpt = row
    
    # Remove some unimportant difference, might not be needed for untrained data
    #original_digar = normalize(orig) 
    #pro_norm = normalize(pro) 
    #gpt_norm = normalize(gpt)
    gpt_norm = ""
    pro_norm = ""
    original_digar = orig
    modified_orig = original_digar.replace("w", "v").replace("W", "V")
    modified_orig = normalize_ocr_text(modified_orig)
    if not pro:
        errorc_pro = 9999
        errorc_pro_modern = 9999
    else:
        pro_norm = pro
        modified_pro= pro_norm.replace("w", "v").replace("W", "V")
        modified_pro = normalize_ocr_text(modified_pro)
        errorc_pro = cer(original_digar, pro_norm)
        errorc_pro_modern = cer(modified_orig, modified_pro)
    if not gpt:
        errorc_gpt = 9999
        errorc_gpt_modern = 9999
    
    else:
        gpt_norm = gpt
    # Modernize w -> v
        modified_gpt= gpt_norm.replace("w", "v").replace("W", "V")
    
        modified_gpt = normalize_ocr_text(modified_gpt)
        errorc_gpt = cer(original_digar, gpt_norm)
        errorc_gpt_modern = cer(modified_orig, modified_gpt)
    


    
    
    
    #print(errorc_pro)
    #print(errorc_gpt)
    


    df = pd.DataFrame ([{
            "newspaper": newspaper,
            "block" : block,
            "orig": original_digar,
            "Gemini": pro_norm,
            "GPT": gpt_norm,
            "cer_pro": errorc_pro,
            "ceer_gpt": errorc_gpt,
            "ceer_mod_gpt": errorc_gpt_modern,
            "ceer_mod_pro": errorc_pro_modern,
        }])

    results.append({
        "newspaper": newspaper,
        "block": block,
        "cer_pro": errorc_pro,
        "cer_gpt": errorc_gpt,
        "ceer_mod_gpt": errorc_gpt_modern,
        "ceer_mod_pro": errorc_pro_modern,
        "orig": original_digar,
        "Gemini": pro_norm,
        "GPT": gpt_norm
    })
    
    row = cursor.fetchone()

df = pd.DataFrame(results)

write_conn.register("df", df)

In [ ]:
%%sql
select * FROM block_quality where correct_ind IN( 'C','Y') and newspaper ='kaja19290813-1.1.5' and block = 'TB00006.xml'


In [ ]:
%%sql
SELECT newspaper, block, count(*)
 FROM block_quality ft where correct_ind IN( 'C','Y')


 and line_amt >  1
  group by  1,2 having count(*) > 1
  --group by 1

In [ ]:
%%sql
select 
*
--distinct block
 --source ,count(*) knt 
FROM full_text where newspaper ='postimeesew19390716.1.1' 
--and source = 'anchoring.py'
--group by 1 order by knt desc
and block not like '%TB00%'
--and block ='TB00004.xml'

In [ ]:
%%sql
select *  FROM
block_quality where correct_ind IN( 'C','Y')


and (newspaper, left(block,7)) not in
(
SELECT 
Newspaper,
Block
--Train_Input,
--case when pro is not null then 'Pro' Else 'FLASH' END AS Ind,
--COALESCE(Pro,FLash) AS Train_Output
--Split_blocks_No_Coords as Train_Output
FROM(
SELECT 
Newspaper,
Block,
MAX(CASE WHEN Source = 'split_blocks_pypdf' then trim(text) else null END)  AS Train_Input,
MAX(CASE WHEN Source IN( 'split_blocks_no_coords') then trim(text) else null END)  AS Split_blocks_No_Coords
FROM Full_Text
where source  IN( 'split_blocks_pypdf','split_blocks_no_coords')
--AND Block <> 'All'
AND NOT EXISTS(
    SELECT 1 FROM OLMOCR_TRAIN 
    WHERE OLMOCR_TRAIN.Newspaper = Full_text.newspaper
    AND OLMOCR_TRAIN.BLock = FUll_Text.BLock
)
/*AND EXISTS(
SELECT 1 FROM BLOCK_QUALITY BQ
WHERE BQ.NEwspaper = Full_Text.newspaper
AND left(BQ.block,7) = full_text.block
AND Bq.Correct_Ind = 'Y' 
)*/
GROUP BY Newspaper,Block
having Train_input is not null and Split_blocks_No_Coords is not null
) SQ --  WHERE len(Train_Output) > 0 and len(Train_input) > 0
)

In [ ]:
%%sql

--INSERT INTO block_quality (Newspaper, Block,Correct_ind, Line_amt )
select  
Newspaper, Block, 'C', 0 
FROM(
SELECT 
Newspaper, Block,ceer_mod_pro,ceer_mod_gpt
--case when ceer_mod_pro <= 0.05 and ceer_mod_gpt >  0.05 then 1 else 0 end  as GPT_Negative,
--case when ceer_mod_pro > 0.05 and ceer_mod_gpt <=  0.05 then 1 else 0 end  as Pro_Negative,
--ceer_mod_pro,
--ceer_mod_gpt,
--abs(ceer_mod_gpt - ceer_mod_pro) as diff,
--df.*
    from df
  where  
-- ((ceer_mod_pro <= 0.05 and ceer_mod_gpt >  0.05 ) OR (ceer_mod_pro > 0.05 and ceer_mod_gpt <=  0.05))


   (ceer_mod_pro <= 0.05 OR ceer_mod_gpt <= 0.05)
  )a
  --   order by diff desc

In [ ]:
%%sql
SELECT  * 
FROM Full_Text
WHERE
1=1
--AND timestamp > current_timestamp - INTERVAL 4 HOUR
--and Source = 'gpt-4o'
AND newspaper = 'postimeesew19370426.1.1'
and block = 'TB00048'
limit 5

In [ ]:
from jiwer import wer
from jiwer import cer
import re
from collections import Counter

def normalize(text):
    # Fix hyphenation at end of line (e.g., "soo-\nmaline" → "soomaline")
    text = re.sub(r'-\s*\n', '', text)

    # Replace " - " or other mid-sentence hyphens with space (e.g., "miski - muu" → "miski muu")
    text = re.sub(r'\s*[-–—]\s*', ' ', text)

    # Normalize w → v
    text = text.replace('w', 'v').replace('W', 'V')

    # Remove multiple newlines and whitespace
    text = re.sub(r'\s+', ' ', text)

    return text.strip()

def count_words(text):
    text = re.sub(r'[^\w\s]', '', text)
    return Counter(text.split())

def dict_diff(dict1, dict2):
    diff = {}
    # Keys only in dict1
    for k in dict1:
        if k not in dict2:
            diff[k] = ('only_in_orig', dict1[k])
        elif dict1[k] != dict2[k]:
            diff[k] = ('different_values', dict1[k], dict2[k])
    # Keys only in dict2
    for k in dict2:
        if k not in dict1:
            diff[k] = ('only_in_new', dict2[k])
    return diff


cursor = conn.execute(
    f"""
/* Check for newspapers that do not have text from current model */
SELECT digar.Newspaper, other.Source, other.Type, state.Correct_Percent ,digar.Text AS digar_text, other.Text AS other_text
FROM Full_Text digar
LEFT JOIN Full_Text other
ON digar.Newspaper = other.Newspaper
AND other.Source <> 'dea.digar'
AND LEN(other.Text) > 10
LEFT JOIN state
 ON digar.Newspaper = state.Newspaper
WHERE digar.Source = 'dea.digar'
AND digar.Type = 'corrected'
and other.Source IS NOT NULL
and other.Source = 'split_blocks_pypdf'

    """
)
row = cursor.fetchone()

while row :
    # Original text has all w turned to v and removed all dashes.
    # Detected text needs to have all end of lines removed with the same treatment as original text

    newspaper = row[0]
    source = row[1]
    typedesc = row[2]
    correct_percent = row[3]
    digar_text = row[4]
    other_text = row[5]

    if not all([newspaper, source, typedesc, digar_text, other_text]):
        print(f"Skipping row due to missing values: {row}")
        row = cursor.fetchone()
        continue

    original_digar = normalize(digar_text) # Text from 'dea.digar'
    new_input = normalize(other_text) # Text from the other source



    errorw = wer(original_digar, new_input)
    errorc = cer(original_digar, new_input)
    word_count_orig = count_words(original_digar)
    word_count_new = count_words(new_input)
    per_word_error = round(len(dict_diff(word_count_orig, word_count_new)) / len(word_count_orig) * 100, 1)


    #print(f"Newspaper: {row[0]}")
    #print(f"Initial quality: {row[3]}%")
    #print(f"Error rate by word count: {per_word_error}%")
    #print(f"Word Error Rate (WER): {errorw}")
    #print(f"Character Error Rate (CER): {errorc}")
    #print(dict_diff(word_count_orig,word_count_new))
    #print(original_digar)
    #print("---")
    #print(new_input)
    #print(row[5])
    write_conn.execute("""
        INSERT INTO Validation (Newspaper, Source,Type,CER,WER,Per_Word_Error, Timestamp )
        SELECT ?, ?, ?, ?, ?, ?, CURRENT_TIMESTAMP
        WHERE NOT EXISTS (
            SELECT 1 FROM Validation
            WHERE Newspaper = ?
                  AND Source = ?
                  AND Type = ?

        )
        """, (newspaper, source, typedesc,errorc,errorw,per_word_error,newspaper,source, typedesc))
    
    row = cursor.fetchone()

write_conn.execute("""
        COPY Validation TO 'duckdb/validation.parquet' (FORMAT PARQUET);
        """)

In [ ]:
%%sql 

select * FROM Validation where Source = 'split_blocks_pypdf' limit 1

In [ ]:
conn.execute("SELECT Text FROM Full_Text WHERE Newspaper='sakalaew19361118.1.6' and  source = 'anchoring.py'").fetchone()


In [ ]:
%%sql
SELECT 
*
--Source,
--max(Text,1)
 FROM Full_Text
WHERE Newspaper= 'sakalaew19361118.1.6'
and source IN( 'split_blocks.py','split_blocks_no_coords')
and block ='TB00004'
--and text like '%õisu%'
--group by Source


In [ ]:
from src.utils.anchor_olmocr import get_anchor_text

anchor_text = get_anchor_text("/mnt/3de36453-6164-4568-91b5-ae973509273e/Git/EE-Gothic-Script-OCR/src/datasets/bronze/scanned/sakalaew19361118.1.6/image/sakalaew19361118.1.6.pdf", 1, pdf_engine="pdfreport")

jpg_width = 2479
jpg_height = 3508

row_nbr = 0

lines = []

for row in anchor_text.splitlines():
    if row_nbr < 2:
        if row_nbr == 0:
            page_size_list = row.split(" ")

            x_split = page_size_list[-1].split("x")

            pdf_width = float(x_split[0])
            pdf_height  = float(x_split[1])

            row_nbr += 1
        else:
            row_nbr += 1
    elif row.startswith('['):
        #print(row)
        ct_split = row.split(']')
        #print(ct_split[1])
        coord = ct_split[0]  # Get text before the first ']'
        text = ct_split[1]
        text = text.strip()
        #print(text)
        coord = coord.strip('[')   # Remove the '[' from the start
        coord = coord.split('x') # Output: 12.34, 56.78 etc.

        #x_img = (x_pdf / pdf_width) * img_width
        #_img = img_height - ((y_pdf / pdf_height) * img_height)
        x_img = int(float(coord[0]) / pdf_width *   jpg_width)
        y_img = int(jpg_height - (float(coord[1]) / pdf_height * jpg_height) )
        line = f"{text} [{x_img},{y_img}] \n "
        lines.append(line)

block = ''.join(lines)
print(block)